In [6]:
!pip install jupyter-dash

import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px

# Load data directly from URL
csv_file = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(csv_file)
print(f"✅ Loaded {len(spacex_df)} rows from {csv_file}")

# Payload range for slider
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

# Prepare options for site dropdown
sites = spacex_df['Launch Site'].unique().tolist()
site_options = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in sites]

# Initialize JupyterDash app instead of normal Dash
app = JupyterDash(__name__)

# App layout
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Label("Launch Site:"),
        dcc.Dropdown(
            id='site-dropdown',
            options=site_options,
            value='ALL',
            placeholder="Select a Launch Site...",
            searchable=True
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Payload Range (kg):"),
        dcc.RangeSlider(
            id='payload-slider',
            min=0, max=10000, step=1000,
            value=[min_payload, max_payload],
            marks={0: '0', 2500: '2.5k', 5000: '5k', 7500: '7.5k', 10000: '10k'}
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    html.Br(),
    
    html.Div(id='success-pie-chart', style={'width': '48%', 'display': 'inline-block'}),
    html.Div(id='success-payload-scatter-chart', style={'width': '48%', 'display': 'inline-block'})
])

# Callback for pie chart
@app.callback(
    Output('success-pie-chart', 'children'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', 
                     title='Total Success Launches By Site',
                     color_discrete_map={1: 'green', 0: 'red'})
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class', values='class',
                     title=f"Success Launches for {entered_site}",
                     color_discrete_map={1: 'green', 0: 'red'})
    return dcc.Graph(figure=fig)

# Callback for scatter plot
@app.callback(
    Output('success-payload-scatter-chart', 'children'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    filtered_df = spacex_df[mask]
    
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title='Payload vs Success (All Sites)',
                         hover_data=['Flight Number'])
    else:
        site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(site_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title=f"Payload vs Success ({entered_site})",
                         hover_data=['Flight Number'])
    return dcc.Graph(figure=fig)

# Run app inline in Jupyter notebook
app.run(mode='inline', debug=True, port=8090)


✅ Loaded 56 rows from https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv


C:\Users\hemac\anaconda3\Lib\site-packages\dash\dash.py:642: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

